In [24]:
import pytesseract
import cv2
import difflib
import pandas as pd
import os
import tkinter as tk
from tkinter import *
import re

In [2]:
def cleanup_text(text):
    return "".join([c if ord(c) < 128 else "" for c in text]).strip()

def convert_lower(text):
    return text.lower()

image = cv2.imread("/home/ravindra/Documents/project/assets/list.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# use Tesseract to OCR the image
text = pytesseract.image_to_string(image)
cleaned_text = cleanup_text(text)
final_text = str(convert_lower(cleaned_text))

print(final_text)


ph326051246/26145398
tins27620241762
cash/bill

ho 009742 0 fip 0 09-07-2015

 

 

description qty rate amount
ee
petrol 3-990 7518 300.00

cash 300-00
thark yous.. visit agaih

vv tin!276302417620

ci 07:27:58 h/c ko. 9


In [16]:
def lst_numbers(final_text):
    l = final_text.split()
    lst_num = []
    for line in l:
        lst_num.append("".join(re.findall(r'[-+]?\d*\.\d+|\d+', line)))

    lst_num = list(filter(None, lst_num))
    return lst_num


def get_category(final_text):
    Categories = ['shopping', 'stationary','petrol','tax','medical','Electricity','Dish' , 'Stores','Tax', 'Bakery','Furniture', 'Grocery','Electrical Instruments', 'Food Items','Vegetable Market', 'Jewellery','Super Market', 'Garments','Bazar','Electronic Gadget', 'Cafe','Service','Restaurants', 'Malls','Telephone' ]
    bill = final_text.split()
    billType = difflib.get_close_matches(bill, Categories)
    matched_category = []
    remove_lst = [[]]
    for word in bill:
        extract = difflib.get_close_matches(word.lower(), Categories)
        matched_category.append(extract)

    res = [i for i in matched_category if i not in remove_lst]
    return res[0][0]
txn_category = get_category(final_text)
txn_category

'petrol'

In [15]:
lst_numbers(final_text)

['32605124626145398',
 '27620241762',
 '009742',
 '0',
 '0',
 '09072015',
 '3990',
 '7518',
 '30000',
 '30000',
 '276302417620',
 '072758',
 '9']

In [47]:
# Create object
root = Tk()
root.title('Confirm')
# Adjust size
root.geometry( "300x300" )

#     label.config( text = clicked.get() )
#     print(text)
amount = 0

def display_selected():
    global amount 
    # Change the label text
    label.config( text = clicked.get() )
    amount = clicked.get()
  
# Dropdown menu options
options = lst_numbers(final_text)
  
T = Text(root, height = 4, width = 52)
T.pack()
T.insert(tk.END, Fact)
# datatype of menu text
clicked = StringVar()
sv = options[0]
# initial menu text
clicked.set( options[0] )
  
# Create Dropdown menu
drop = OptionMenu( root , clicked , *options )
drop.pack()

Fact = "Multiple numericals detected. Please select the total amount of the bill"
# Create button, it will change label text
button = Button( root , text = "ok" , command = display_selected).pack()

b2 = Button(root, text = "Exit",
            command = root.destroy).pack() 
 
# Create Label
label = Label( root , text = " " )
label.pack()
# Execute tkinter
root.mainloop()

In [49]:
def find_dates(input):

    date_formats = [ 
                # Matches date format MM/DD/YYYY
                '(\d{2}\/\d{2}\/\d{4})',

                # Matches date format MM-DD-YYYY
                '(\d{2}-\d{2}-\d{4})',

                # Matches date format YYYY/MM/DD
                '(\d{4}\/\d{1,2}\/\d{1,2})',

                # Matches ISO 8601 format (YYYY-MM-DD)
                '(\d{4}-\d{1,2}-\d{1,2})',


                # Matches full_month_name dd, YYYY or full_month_name dd[suffixes], YYYY
                '(January|February|March|April|May|June|July|August|September|October|November|December)(\s\d{1,2}\W\s\d{4}|\s\d(st|nd|rd|th)\W\s\d{4})',

                # Matches abbreviated_month_name dd, YYYY or abbreviated_month_name dd[suffixes], YYYY
                '(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept|Oct|Nov|Dec)(\s\d{1,2}\W\s\d{4}|\s\d(st|nd|rd|th)\W\s\d{4})',

                # Matches ISO 8601 format with time and time zone
                # yyyy-mm-ddThh:mm:ss.nnnnnn+|-hh:mm
                '\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(\+|-)\d{2}:\d{2}',


                # Matches ISO 8601 format Datetime with timezone
                # yyyymmddThhmmss+|-hhmm
                '\d{8}T\d{6}(\+|-)\d{4}' , 

                ''
                ]

    for item in date_formats:
        date_format = re.compile(r'\b{}\b'.format(item), re.IGNORECASE|re.MULTILINE)
        find_date = re.search(date_format, input)

        if find_date:
            return find_date.group(0)



txn_date = find_dates(final_text)

In [48]:
# if file does not exist write header 
csv_file_name = './report/budget_report.csv'
column_names = ['date', 'category', 'total_bill']

df = pd.DataFrame(columns=column_names)

row_value = {'date': txn_date , 'category': txn_category, 'total_bill': amount}

df = df.append(row_value, ignore_index = True)

df.to_csv(csv_file_name, mode='a', header=False, index = False)
